# なんかのこびとをつくる

In [22]:
# Here is the updated code to include the new site for scraping:

import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Ensure directories exist
if not os.path.isdir("pic"):
    os.makedirs("pic")
if not os.path.isdir("name"):
    os.makedirs("name")

# URLs to scrape
urls = ["https://wikiwiki.jp/kobito/", "https://www.kobitos.com/kobitos", "https://www.1999.co.jp/10156056"]

for url in urls:
    # Send a GET request
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Save text data
    text = soup.get_text()
    with open(os.path.join("name", f"{url.replace('/', '_')}_text.txt"), "w", encoding='utf-8') as f:
        f.write(text)

    # Save image data
    for img in soup.find_all("img"):
        img_url = img.attrs.get("src")
        if img_url is None:
            continue
        if ".jpg" in img_url or ".webp" in img_url:
            # If the image url is relative, prepend the base url (to make it absolute)
            img_url = urljoin(url, img_url)
            # Use the basename of the URL (the last part after /) as the filename
            filename = os.path.join("pic", os.path.basename(img_url))
            # Send a GET request to download the image
            img_data = requests.get(img_url).content
            with open(filename, "wb") as f:
                f.write(img_data)


In [23]:
from PIL import Image, ImageDraw, ImageFont
import os
import random

# "pic"フォルダ内の全ての画像ファイル名を取得
image_files = [f for f in os.listdir("pic") if f.endswith(('.jpg', '.webp'))]

# ランダムに2つの画像を選択
img1 = Image.open(os.path.join("pic", random.choice(image_files)))
img2 = Image.open(os.path.join("pic", random.choice(image_files)))

# 2つの画像を同じサイズにリサイズ
img1 = img1.resize((320, 320))
img2 = img2.resize((320, 320))

# 2つ目の画像を半透明にする
img2 = img2.convert("RGBA")
for x in range(img2.width):
    for y in range(img2.height):
        r, g, b, a = img2.getpixel((x, y))
        img2.putpixel((x, y), (r, g, b, int(a * 0.5)))

# 2つの画像を重ね合わせる
composite = Image.alpha_composite(img1.convert("RGBA"), img2)

# ユーザがこびとの名前を入力
kobito_name = input("こびとの名前を入力してください: ")

# テキストを追加する
draw = ImageDraw.Draw(composite)
font = ImageFont.truetype("arial.ttf", 15)  # フォントとサイズを指定
draw.text((10, 10), kobito_name, fill="black", font=font)  # テキストの位置、内容、色、フォントを指定

# 合成画像を表示
composite.show()


こびとの名前を入力してください:  ゆういこびと
